# Feature engineering, xgboost
https://www.kaggle.com/dlarionov/feature-engineering-xgboost


-　DateNum作成 'date_block_num',→YYYYMMで代用
- 祝日フラグ
- month
- days
- アイテムカテゴリ（デフォルト)-> カテゴリCategory、ジャンルType
- 月の売り上げ
- 月の売り上げラグ 1,2,3,6,12
- アイテムあたりの「週Or日」平均販売個数 1,2,3,6,12　　'date_item_avg_item_cnt_lag_1',
- ショップあたり「週Or日」平均販売個数ラグ1，2，3，6，12　　'date_shop_avg_item_cnt_lag_1',
- アイテムカテゴリあたり「週Or日」平均販売個数ラグ1,2,3,6,12'date_cat_avg_item_cnt_lag_1',
- カテゴリあたり「週Or日」平均販売個数ラグ1,2,3,6,12，'date_cat_avg_item_cnt_lag_1',
- ジャンル　あたり[月] 平均販売個数ラグ1,2,3,6,12
- アイテムカテゴリ/ショップあたり[月] 平均販売個数ラグ1,2,3,6,12
- カテゴリ/ショップあたり[月] 平均販売個数ラグ1,2,3,6,12
- ジャンル/ショップ　あたり[月] 平均販売個数ラグ1,2,3,6,12
- ASP　月当たり
- 価格下落率
- 最終販売日
- 初販売日
- ショップあたりの最終販売日
- ショップあたりの初販売日
    'delta_price_lag',


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib inline

In [2]:
category_names = pd.read_csv('category_names.csv')
category_names.columns = ['ItemCategoryId', 'ItemCategoryName']

item_categories = pd.read_csv('item_categories.csv')
item_categories.columns = ['ItemId', 'ItemCategoryId']

sales_history = pd.read_csv('sales_history.csv')
sales_history.columns = ['Date', 'StoreId', 'ItemId', 'ItemPrice', 'SoldNum']
sales_history['Date'] = pd.to_datetime(sales_history['Date'])

test = pd.read_csv('test.csv')
test.columns = ['index', 'ItemId', 'StoreId']
#item_categories['ItemCategoryId'].unique()
#category_names.query('ItemCategoryName.str.contains("")')

'''全アイテムとジャンル分け・休日フラグ付きデータ'''
sales_total = pd.read_csv('./csv/sales_total.csv')
'''170品目のリスト'''
test_item = test['ItemId'].unique()
test_item = pd.DataFrame({'ItemId':test_item})

'''年月のリスト'''
yyyymm = pd.read_csv('./csv/yyyymm.csv')

'''170品目と月番号加工済みデータ '''
#target = pd.read_csv('./csv/sales_170_date_block.csv')
target = pd.read_csv('./csv/sales_170_date_block_asp_rev.csv')
#test_item
'''カテゴリ名と連結'''
ttest = pd.merge(test,item_categories, on='ItemId')
ttest = pd.merge(ttest, category_names,on='ItemCategoryId')
print('予測データの品目数: ',ttest['ItemId'].nunique(),'\n',
      '訓練データの品目数: ',sales_history['ItemId'].nunique(),'\n',
      '予測データのカテゴリ数: ',item_categories['ItemCategoryId'].nunique(),'\n',
      '訓練データのカテゴリ数: ',ttest['ItemCategoryId'].nunique(),'\n',
     '売上の総数: ',sales_history['SoldNum'].sum(), '\n',
     '')

予測データの品目数:  170 
 訓練データの品目数:  9040 
 予測データのカテゴリ数:  26 
 訓練データのカテゴリ数:  26 
 売上の総数:  1262869.0 
 


In [3]:
sales_total

,Unnamed: 0,Date,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,CatName,TypeName,Category,Type,is_holiday,YYYYMM,YYYY,MM,date_block_num,Revenue
0,0,2018-01-01,9,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,1,0,420.0
1,1,2018-01-01,6,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,1,0,420.0
2,2,2018-01-01,10,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,1,0,420.0
3,3,2018-01-02,10,1000001,420,2.0,100,映画,DVD,5,1,0,201801,2018,1,0,840.0
4,4,2018-01-02,9,1000001,420,1.0,100,映画,DVD,5,1,0,201801,2018,1,0,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119565,1119565,2019-10-20,6,3300319,3400,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,3400.0
1119566,1119566,2019-10-20,0,3300319,3400,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,3400.0
1119567,1119567,2019-10-25,0,3300323,9350,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,9350.0
1119568,1119568,2019-10-28,1,3300325,2120,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,2120.0


In [4]:
sales_total = sales_total.drop(['Unnamed: 0'],axis=1)

In [5]:
sales_total

,Date,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,CatName,TypeName,Category,Type,is_holiday,YYYYMM,YYYY,MM,date_block_num,Revenue
0,2018-01-01,9,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,1,0,420.0
1,2018-01-01,6,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,1,0,420.0
2,2018-01-01,10,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,1,0,420.0
3,2018-01-02,10,1000001,420,2.0,100,映画,DVD,5,1,0,201801,2018,1,0,840.0
4,2018-01-02,9,1000001,420,1.0,100,映画,DVD,5,1,0,201801,2018,1,0,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119565,2019-10-20,6,3300319,3400,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,3400.0
1119566,2019-10-20,0,3300319,3400,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,3400.0
1119567,2019-10-25,0,3300323,9350,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,9350.0
1119568,2019-10-28,1,3300325,2120,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,2120.0


In [6]:
sales_total['Date'] = pd.to_datetime(sales_total['Date'])

In [7]:
sales_total['YYYYMM'] = sales_total['Date'].dt.strftime('%Y%m')
sales_total['YYYY'] = sales_total['Date'].dt.strftime('%Y')
sales_total['MM'] = sales_total['Date'].dt.strftime('%m')

In [8]:
sales_total

,Date,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,CatName,TypeName,Category,Type,is_holiday,YYYYMM,YYYY,MM,date_block_num,Revenue
0,2018-01-01,9,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,01,0,420.0
1,2018-01-01,6,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,01,0,420.0
2,2018-01-01,10,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,01,0,420.0
3,2018-01-02,10,1000001,420,2.0,100,映画,DVD,5,1,0,201801,2018,01,0,840.0
4,2018-01-02,9,1000001,420,1.0,100,映画,DVD,5,1,0,201801,2018,01,0,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119565,2019-10-20,6,3300319,3400,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,3400.0
1119566,2019-10-20,0,3300319,3400,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,3400.0
1119567,2019-10-25,0,3300323,9350,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,9350.0
1119568,2019-10-28,1,3300325,2120,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,2120.0


In [9]:
sales_total['Revenue'] = sales_total['SoldNum'] * sales_total['ItemPrice']

In [10]:
sales_total

,Date,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,CatName,TypeName,Category,Type,is_holiday,YYYYMM,YYYY,MM,date_block_num,Revenue
0,2018-01-01,9,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,01,0,420.0
1,2018-01-01,6,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,01,0,420.0
2,2018-01-01,10,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,01,0,420.0
3,2018-01-02,10,1000001,420,2.0,100,映画,DVD,5,1,0,201801,2018,01,0,840.0
4,2018-01-02,9,1000001,420,1.0,100,映画,DVD,5,1,0,201801,2018,01,0,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119565,2019-10-20,6,3300319,3400,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,3400.0
1119566,2019-10-20,0,3300319,3400,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,3400.0
1119567,2019-10-25,0,3300323,9350,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,9350.0
1119568,2019-10-28,1,3300325,2120,1.0,330,音楽,レコード,7,17,0,201910,2019,10,21,2120.0


In [11]:
target = pd.merge(sales_total,test_item,on='ItemId',how='right')

In [12]:
#target.to_csv('./csv/target170items.csv')

In [13]:
target_monthly = target.groupby(['date_block_num','YYYYMM','YYYY','MM',
    'StoreId','ItemId','ItemCategoryId','Category','Type']).agg({
    'SoldNum':'sum','is_holiday':'sum','ItemPrice':'mean','Revenue':'sum'
}).reset_index()

In [14]:
target_monthly

,date_block_num,YYYYMM,YYYY,MM,StoreId,ItemId,ItemCategoryId,Category,Type,SoldNum,is_holiday,ItemPrice,Revenue
0,0,201801,2018,01,0,1000001,100,5,1,6.0,0,420.0,2520.0
1,0,201801,2018,01,0,1000002,100,5,1,2.0,0,250.0,500.0
2,0,201801,2018,01,0,1000004,100,5,1,1.0,0,120.0,120.0
3,0,201801,2018,01,0,1000005,100,5,1,8.0,0,220.0,1760.0
4,0,201801,2018,01,0,1000006,100,5,1,2.0,0,190.0,380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25402,21,201910,2019,10,17,2900075,290,1,22,6.0,0,590.0,3540.0
25403,21,201910,2019,10,17,3200001,320,4,10,1.0,0,3720.0,3720.0
25404,21,201910,2019,10,17,3200026,320,4,10,4.0,0,850.0,3400.0
25405,21,201910,2019,10,17,3400001,340,4,9,1.0,1,850.0,850.0


In [21]:
#target_monthly.to_csv('./csv/sales_170_date_block_asp_rev.csv',index=False)

# pivotせずに特徴量作成

In [16]:
target

,Date,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,CatName,TypeName,Category,Type,is_holiday,YYYYMM,YYYY,MM,date_block_num,Revenue
0,2018-01-01,9,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,01,0,420.0
1,2018-01-01,6,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,01,0,420.0
2,2018-01-01,10,1000001,420,1.0,100,映画,DVD,5,1,1,201801,2018,01,0,420.0
3,2018-01-02,10,1000001,420,2.0,100,映画,DVD,5,1,0,201801,2018,01,0,840.0
4,2018-01-02,9,1000001,420,1.0,100,映画,DVD,5,1,0,201801,2018,01,0,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48216,2019-10-02,4,3500001,420,1.0,350,本,オーディオブック_タイプB,6,6,0,201910,2019,10,21,420.0
48217,2019-10-02,0,3500001,420,1.0,350,本,オーディオブック_タイプB,6,6,0,201910,2019,10,21,420.0
48218,2019-10-21,13,3500001,420,1.0,350,本,オーディオブック_タイプB,6,6,0,201910,2019,10,21,420.0
48219,2019-10-25,13,3500001,420,1.0,350,本,オーディオブック_タイプB,6,6,0,201910,2019,10,21,420.0


# Lag関数

In [17]:
def lag_feature(df,lags,col):
    tmp = df[['date_block_num','StoreId','ItemId',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','StoreId','ItemId',col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df,shifted, on=['date_block_num','StoreId','ItemId'],how='left')
    return df

# 売上月次推移

In [18]:
ts = time.time()
matrix = lag_feature(target,[1,2,3,6,12],'SoldNum')
time.time() - ts

36.61730170249939

# 品目あたりの月平均売上個数

In [19]:
group = matrix.groupby(['date_block_num']).agg({'SoldNum':'mean'})
group.columns = ['date_avg_item_cnt']
group.reset_index(inplace=True)

In [20]:
matrix = pd.merge(matrix,group,on='date_block_num',how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix,[1,2,3,6,12],'date_avg_item_cnt')
matrix.drop(['date_avg_item_cnt'],axis=1,inplace=True)

MemoryError: Unable to allocate 554. TiB for an array with shape (76106100912580,) and data type int64

# 店舗あたりの月平均売上個数

In [ ]:
group = matrix.groupby(['date_block_num','StoreId']).agg({'SoldNum':'mean'})
group.columns = ['date_shop_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=['date_block_num','StoreId'],how='left')
matrix['date_shop_avg_item_cnt'] = matrix['date_shop_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix,[1,2,3,6,12],'date_shop_avg_item_cnt')
matrix.drop(['date_shop_avg_item_cnt'],axis=1,inplace=True)

# アイテムカテゴリあたりの月平均売上個数

In [ ]:
group = matrix.groupby(['date_block_num','ItemCategoryId']).agg({'SoldNum':'mean'})
group.columns = ['date_itemcat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=['date_block_num','ItemCategoryId'],how='left')
matrix['date_itemcat_avg_item_cnt'] = matrix['date_itemcat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix,[1,2,3,6,12],'date_itemcat_avg_item_cnt')
matrix.drop(['date_itemcat_avg_item_cnt'],axis=1,inplace=True)

# カテゴリあたりの月平均売上個数

In [ ]:
group = matrix.groupby(['date_block_num','Category']).agg({'SoldNum':'mean'})
group.columns = ['date_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=['date_block_num','Category'],how='left')
matrix['date_cat_avg_item_cnt'] = matrix['date_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix,[1,2,3,6,12],'date_cat_avg_item_cnt')
matrix.drop(['date_cat_avg_item_cnt'],axis=1,inplace=True)

# ジャンルあたりの月平均売上個数

In [ ]:
group = matrix.groupby(['date_block_num','Type']).agg({'SoldNum':'mean'})
group.columns = ['date_type_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=['date_block_num','Type'],how='left')
matrix['date_type_avg_item_cnt'] = matrix['date_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix,[1,2,3,6,12],'date_type_avg_item_cnt')
matrix.drop(['date_type_avg_item_cnt'],axis=1,inplace=True)

# アイテムカテゴリ・ショップあたりの月平均売上個数

In [ ]:
group = matrix.groupby(['date_block_num','StoreId','ItemCategoryId']).agg({'SoldNum':'mean'})
group.columns = ['date_shop_itemcat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=['date_block_num','StoreId','ItemCategoryId'],how='left')
matrix['date_shop_itemcat_avg_item_cnt'] = matrix['date_shop_itemcat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix,[1,2,3,6,12],'date_shop_itemcat_avg_item_cnt')
matrix.drop(['date_shop_itemcat_avg_item_cnt'],axis=1,inplace=True)

# カテゴリ・ショップあたりの月平均売上個数

In [ ]:
group = matrix.groupby(['date_block_num','StoreId','Category']).agg({'SoldNum':'mean'})
group.columns = ['date_shop_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=['date_block_num','StoreId','Category'],how='left')
matrix['date_shop_cat_avg_item_cnt'] = matrix['date_shop_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix,[1,2,3,6,12],'date_shop_cat_avg_item_cnt')
matrix.drop(['date_shop_cat_avg_item_cnt'],axis=1,inplace=True)

# ジャンル・ショップあたりの月平均売上個数

In [ ]:
group = matrix.groupby(['date_block_num','StoreId','Type']).agg({'SoldNum':'mean'})
group.columns = ['date_shop_type_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group,on=['date_block_num','StoreId','Type'],how='left')
matrix['date_shop_type_avg_item_cnt'] = matrix['date_shop_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix,[1,2,3,6,12],'date_shop_type_avg_item_cnt')
matrix.drop(['date_shop_type_avg_item_cnt'],axis=1,inplace=True)

In [ ]:
matrix.fillna(0)

In [ ]:
matrix_corr = matrix.corr()

In [ ]:
sns.heatmap(matrix_corr)